In [5]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [22]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/LMDZ_Data/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values

zg=ZData.zg.rename({'time_counter':'time'})
zg.time
datetimeindex = zg.indexes['time'].to_datetimeindex()
datetimeindex
zg['time'] = datetimeindex
zg.values

<ipython-input-22-ed357fc1a6e7>:9: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = zg.indexes['time'].to_datetimeindex()


array([[[ 0.43451262,  0.23016365, -0.01457122, ...,  0.4902693 ,
          0.49567273,  0.4836069 ],
        [ 0.55367917,  0.34041205, -0.0047477 , ...,  0.45752278,
          0.53351873,  0.56261206],
        [ 0.65311646,  0.403242  ,  0.08700774, ...,  0.50060356,
          0.5714051 ,  0.6233582 ],
        ...,
        [-0.03867773, -0.02485427, -0.01105849, ..., -0.06824062,
         -0.05799268, -0.05017973],
        [-0.02968083, -0.02710977, -0.02449816, ..., -0.03287569,
         -0.03290966, -0.0317592 ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ]],

       [[ 0.69104326,  0.51730365,  0.3375696 , ...,  0.9650228 ,
          0.86958873,  0.8071369 ],
        [ 0.699746  ,  0.64577556,  0.42254966, ...,  0.99530333,
          0.8812564 ,  0.8003175 ],
        [ 0.6857013 ,  0.6389052 ,  0.68703663, ...,  0.99432105,
          0.89786595,  0.812567  ],
        ...,
        [ 0.05608113,  0.05626868,  0.05827835, ...,  

In [23]:


t1=zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [25]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': zg.time})

norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': zg.time})


(2545,)


In [26]:
AO_ds.to_netcdf('LMDZ-AOindex-NDJF-Daily-1980-2014.nc')

In [27]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(2.033478, dtype=float32)
Coordinates:
    time     datetime64[ns] 1985-01-31T12:00:00

In [28]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [29]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-2.645541, dtype=float32)
Coordinates:
    time     datetime64[ns] 1999-11-02T12:00:00

In [ ]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [34]:
zLevData = xr.open_dataset('../../../Data/LMDZ_Data/LMDZOR-HIST-geop_1978_2005_NDJ_1D_histday.nc')
zLevData.presnivs

<xarray.DataArray 'presnivs' (presnivs: 19)>
array([100426.53   ,  98327.56   ,  95346.5    ,  90966.84   ,  84776.86   ,
        76536.46   ,  66292.22   ,  54559.258  ,  42501.758  ,  31805.982  ,
        23787.521  ,  18252.748  ,  13995.969  ,  10320.797  ,   7191.0527 ,
         4661.7383 ,   2732.8513 ,   1345.6179 ,    388.24335], dtype=float32)
Coordinates:
  * presnivs  (presnivs) float32 1.004e+05 9.833e+04 ... 1.346e+03 388.2
Attributes:
    standard_name:  air_pressure
    long_name:      Vertical levels
    units:          Pa
    positive:       down
    axis:           Z

In [42]:
z500=zLevData.geop.sel(presnivs=50000,method='nearest')

z500=z500.rename({'time_counter':'time'})
z500.time
datetimeindex = z500.indexes['time'].to_datetimeindex()
datetimeindex

z500['time'] = datetimeindex
z500.time
z500.values=z500.values/9.81 # geopotential to geopotential height

<ipython-input-42-ec3d5f43d5a9>:5: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = z500.indexes['time'].to_datetimeindex()


In [43]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [45]:
#z145E40N

In [46]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [49]:
#z20E55N

In [50]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [51]:
#z75E55N

In [52]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [53]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [54]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [55]:
EUVal

<xarray.DataArray 'geop' (time: 2545)>
array([ 0.08914706,  0.29263858,  0.09749384, ..., -0.59330146,
       -0.98372817, -1.03098412])
Coordinates:
    presnivs   float32 5.456e+04
  * time       (time) datetime64[ns] 1978-11-01T12:00:00 ... 2005-12-31T12:00:00
    dayofyear  (time) int64 305 306 307 308 309 310 ... 360 361 362 363 364 365
    lon        float32 144.7

In [57]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': z500.time})

In [58]:
EU_ds.to_netcdf('LMDZ-EUindex-NDJF-Daily-1980-2014.nc')